# 3.1 Using Pretrained Word Embeddings to Find Word Similarity 
  
Problem: You need to find out whether two words are similar but not equal.  
  
Solution: Use a pretrained word embedding model. In the code example we use `genism`, a Python library for topic modeling.  

First step is to aquire a pretrained model, here we use Google News. 

In [1]:
%matplotlib inline

import os
from keras.utils import get_file
import gensim
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
figsize(10, 10)

from sklearn.manifold import TSNE
import json
from collections import Counter
from itertools import chain

/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MODEL = 'GoogleNews-vectors-negative300.bin'
path = get_file(MODEL + '.gz', 'https://deeplearning4jblob.blob.core.windows.net/resources/wordvectors/%s.gz' % MODEL)
if not os.path.isdir('generated'):
    os.mkdir('generated')

unzipped = os.path.join('generated', MODEL)
if not os.path.isfile(unzipped):
    with open(unzipped, 'wb') as fout:
        zcat = subprocess.Popen(['zcat'],
                          stdin=open(path),
                          stdout=fout
                         )
        zcat.wait()

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format(unzipped, binary=True)

Once model has finished loading we can use it to find similar words:

In [5]:
model.most_similar(positive=['espresso'])

[('cappuccino', 0.6888186931610107),
 ('mocha', 0.6686208844184875),
 ('coffee', 0.6616826057434082),
 ('latte', 0.6536753177642822),
 ('caramel_macchiato', 0.6491268873214722),
 ('ristretto', 0.6485545635223389),
 ('espressos', 0.6438628435134888),
 ('macchiato', 0.6428250074386597),
 ('chai_latte', 0.6308028697967529),
 ('espresso_cappuccino', 0.6280542612075806)]

##### Discussion: Word embeddings associate an n-dimensional vector with each word in the vocabulary in a way that similar words are near each other. Finding similar words is a mere **nearest-neighbor** search. The Word2vec embeddings are obtained by training a NN to predict a word from its context. Words that can be inserted into similar patterns will get vectors that are close to each other. Here we don't care about the actual task, just about the assigned weights, which we get as a side effect of training the network.  
  
Note later we will see how word embeddings can also be used to feed words into a neural network. It is much more feasible to feed a 300-dim embedding vector into a network than a 3-million-dim one that is one-hot encoded. Moreover, a network fed with pretrained word embeddings doesn't have to learn the relationships between the wrods, but can start with the real task at hand immediately. 

# 3.2 Word2vec Math  

#### Problem: How can you automatically answer questions of the form "A is to B as C is to what?"  
  
#### Solution: Use the semantic properties of the Word2vec model. The `gensim` library makes this straightforward: